In [1]:
import json
from sklearn.model_selection import train_test_split

# Load the dataset
with open('NER_TRAIN_JUDGEMENT.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# print('Number of sentences in the dataset:', len(dataset))

In [ ]:
# Split the dataset into training and validation sets with an 85:15 ratio (randomly stratified)
train_data, val_data = train_test_split(dataset, test_size=0.15, random_state=42)

In [ ]:
# Save the split datasets into separate JSON files
with open('train_dataset.json', 'w', encoding='utf-8') as train_file:
    json.dump(train_data, train_file, ensure_ascii=False, indent=4)

with open('val_dataset.json', 'w', encoding='utf-8') as val_file:
    json.dump(val_data, val_file, ensure_ascii=False, indent=4)

# print("Training set size:", len(train_data))
# print("Validation set size:", len(val_data))

In [ ]:
import json
from collections import defaultdict

# Load the dataset
with open('NER_TRAIN_JUDGEMENT.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Define a function to assign BIO labels
def assign_bio_labels(text, annotations, num):
    # Tokenize the text based on spaces
    tokens = text.split()

    # Initialize a list to store BIO labels for each token
    bio_labels = ['O'] * len(tokens)

    # Iterate through annotations and update BIO labels
    for annotation in annotations:
        for result in annotation['result']:
            start = result['value']['start']
            end = result['value']['end']
            label = result['value']['labels'][0]
            for i, token in enumerate(tokens):
                print(num, token, start, end, text.find(token), text.find(token) + len(token))
                if start == text.find(token) and text.find(token) + len(token) <= end+1:
                    bio_labels[i] = 'B_' + label
                    # print(bio_labels[i],token)
                elif start < text.find(token) + len(token) <= end+1:
                    bio_labels[i] = 'I_' + label
                    # print(bio_labels[i],token)
    return bio_labels

# Initialize a dictionary to store the output
output = defaultdict(dict)
num=0
# Process each data instance
for instance in data:
    case_id = instance['id']
    text = instance['data']['text']
    annotations = instance['annotations']
    bio_labels = assign_bio_labels(text, annotations, num)
    
    # Store the text and BIO labels in the output dictionary
    output[case_id]['text'] = text
    output[case_id]['labels'] = bio_labels
    num+=1

# # Print the output in the specified format
# for case_id, case_data in output.items():
#     text = case_data['text']
#     labels = case_data['labels']
#     print(f'"{case_id}":{{')
#     print(f'    "text": "{text}",')
#     print(f'    "labels": {labels}')
#     print('},')
    
# Save the output to a JSON file
with open('bio_labels.json', 'w', encoding='utf-8') as file:
    json.dump(output, file, ensure_ascii=False, indent=4)
    